In [2]:
!pip install --upgrade datasets

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [24]:
from datasets import load_dataset, Dataset
from huggingface_hub import login
import json

# Load dataset
dataset = load_dataset('lmarena-ai/webdev-arena-preference-10k', split='test')
def transform_conversation_to_alpaca(conversation):
    """
    Transform conversation format to Alpaca format
    """


    # Extract user message and assistant response
    instruction = None
    user_message = None
    assistant_response = None

    for turn in conversation:
        if turn["role"] == "user":
            # Extract text from content
            user_message = turn["content"][0]["text"]
        elif turn["role"] == "assistant":
            # Extract text response and code if available
            assistant_text = turn["content"][0]["text"]

            # If there's code in the object, include it
            if turn.get("object") and turn["object"].get("code"):
                code = turn["object"]["code"]
                assistant_response = f"{assistant_text}\n\n``````{code}``````\n"
            else:
                assistant_response = assistant_text

    # Create Alpaca format entry
    if user_message and assistant_response:
        return {
            "instruction": user_message,
            "input": user_message,
            "output": assistant_response
        }

transformed_alpaca_dataset =[]
def process_row(row):
    winner = row['winner']
    conversation_key = 'conversation_a' if winner == row['model_a'] else 'conversation_b'
    conversation = row[conversation_key]
    transformed_data= transform_conversation_to_alpaca(conversation)
    return(transformed_data)


# Transform your data
transformed_alpaca_dataset = dataset.map(process_row, remove_columns=dataset.column_names)
# print(transformed_alpaca_dataset)
# login()  # Run this and enter your token when prompted
print(transformed_alpaca_dataset)

transformed_alpaca_dataset.push_to_hub("Rajat1327/webdev-llama3-alpaca-formatted")

Map:   0%|          | 0/10501 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 10497
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Rajat1327/webdev-llama3-alpaca-formatted/commit/529aa2d552ee0faec3c96984e3a9c3a05c34b06d', commit_message='Upload dataset', commit_description='', oid='529aa2d552ee0faec3c96984e3a9c3a05c34b06d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Rajat1327/webdev-llama3-alpaca-formatted', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Rajat1327/webdev-llama3-alpaca-formatted'), pr_revision=None, pr_num=None)